In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import csv
# import plotly.express as px

In [8]:
def remove_invalid_values(csv_file):
    # Read the CSV file and filter out lines containing "--,--"
    with open(csv_file, 'r') as file:
        lines = file.readlines()
        filtered_lines = [line for line in lines if "--,--" not in line]

    # Write the filtered lines back to the CSV file
    with open(csv_file, 'w') as file:
        for line in filtered_lines:
            file.write(line)

def sort_csv(csv_file):
    # Read the CSV file into a list of dictionaries
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        csv_data = list(reader)
    
    # Sort the data based on the values in the "B" column
    sorted_data = sorted(csv_data, key=lambda x: float(x['B']))
    
    # Write the sorted data back to the CSV file
    with open(csv_file, 'w', newline='') as file:
        fieldnames = reader.fieldnames
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(sorted_data)

def get_csv_paths(folder_path):
    csv_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                csv_paths.append(os.path.join(root, file))
    return csv_paths

def read_csv_to_dataframes(csv_paths):
    """
    Read CSV files into pandas DataFrames and store them in an array.
    """
    dataframes = []
    for path in csv_paths:
        dataframe = pd.read_csv(path)
        dataframe.sort_values(by=dataframe.columns[0])
        dataframes.append(dataframe)
    return dataframes

def offset_csv_to_max(filename):
    # Step 1: Read CSV and parse contents
    data = pd.read_csv(filename)

    # Step 2: Find the absolute maximum in the y-axis (assuming the CSV is 2-D)
    absolute_max_y = data.iloc[:, 1].abs().max()

    # Step 3: Find the x-value corresponding to the absolute maximum y-value
    max_x_index = data.iloc[:, 1].abs().idxmax()
    x_value_at_max_y = data.iloc[max_x_index, 0]

    # Step 4: Calculate the offset to move the absolute maximum to x = 0
    offset = x_value_at_max_y

    # Step 5: Adjust x-coordinate of all points
    data.iloc[:, 0] -= offset

    # Step 6: Save modified DataFrame back to the original CSV file
    data.to_csv(filename, index=False)

    print("CSV offset successfully. File overwritten with new data.")

def plot_and_save_dataframes(folder_name, dataframes):
    """
    Plot and save DataFrames under the specified folder.
    Assumes the x-axis is the first column and the y-axis is the second column.
    """
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for idx, df in enumerate(dataframes):
        # Check if DataFrame has at least two columns
        if len(df.columns) >= 2:
            x = df["B"]
            y = df["I_c"]
            plt.figure()
            plt.plot(x, y)
            plt.xlabel('X-axis')
            plt.ylabel('Y-axis')
            plt.title(f'Plot {idx+1}')
            plt.savefig(os.path.join(folder_name, f'plot_{idx+1}.png'))
            plt.close()
        else:
            print(f"DataFrame {idx+1} does not have sufficient columns to plot.")

In [10]:
array_of_csvs = get_csv_paths(r"..\experimental data\experimental csvs")
# offset_csv_to_max(r"../experimental data\experimental csvs\5.0uV.csv")
# for str in array_of_csvs:
#     sort_csv(str)

In [11]:
dataframes = read_csv_to_dataframes(array_of_csvs)

In [11]:
# fig = px.line(dataframes[0], x='B', y='I_c')
# fig.show()

In [12]:
plot_and_save_dataframes("test", dataframes)